In [ ]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import os


In [ ]:
#def crop_it(feed):
feed = ['/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video5/DLC_GazeXBI-anc-2023-05-02/videos/croppedDLC_resnet50_DLC_GazeXBIMay2shuffle1_1030000_filtered.h5',
        '/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video5/DLC_GazeXBI-anc-2023-05-02/videos/croppedDLC_resnet50_DLC_GazeXBIMay2shuffle1_1030000_filtered_labeled.mp4',
        'DLC_resnet50_DLC_GazeXBIMay2shuffle1_1030000']
fl_pth, vid_pth, x = feed
# deeplabcut.analyze_videos(config_path, 'CropGr.mp4', save_as_csv=True, dynamic=(True,.6,30))
print("Starting Cropping")

# Define the names of the facial features that you want to extract frames for
feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

# Load tracking results generated by DeepLabCut
tracking_data = pd.read_hdf(fl_pth)
# x = tracking_data['DLC_resnet50_DLCApr12shuffle1_1030000'][feature_names]
# x.isna().any()

necessary_points = tracking_data['DLC_resnet50_DLC_GazeXBIMay2shuffle1_1030000'][feature_names]
# Define the threshold for the confidence score of the facial features
confidence_threshold = 0.8

# Load the input video
cap = cv2.VideoCapture(vid_pth)
 
tot_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# initializing the frame id list
labels = []
tot_yes = 0
frame_id=[]

# Loop through the video frames and extract frames with facial features
frame_number = 0
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # Get the tracking data for the current frame
        frame_data = necessary_points.loc[frame_number]
        # print(frame_data)

        # Check if the desired facial features are present in the frame
        feature_present = False
        check = 0            
        for feature_name in feature_names:
            if feature_name in frame_data and frame_data.loc[feature_name].loc['likelihood'] > confidence_threshold:
                check += 1
        if check==4:
            feature_present = True

        # If the desired facial features are present, save the frame to the output video
        if feature_present: #check==4
            tot_yes += 1
            labels.append([frame_number, 1])
            frame_id.append(frame_number)
        else:
            labels.append([frame_number, 0])

        # Display the output
        cv2.imshow('output', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        print(round(frame_number/tot_frame*100, 2), end = '\r')
        # Increment the frame number
        frame_number += 1
    else:
        break

labels.append([tot_yes, (frame_number-tot_yes)])##this list contains the total number of frames with faces (tot_yes) and the total number of frames without faces (frame_index-tot_yes). 
labels.append([frame_id,0])
text_file = open("text_fileFacehuman_DLCds.csv", "w")#opens a new file called "text_fileFacehumands.csv" in write mode (`"w"`). This file will be used to store the face detection information.
labelled = pd.DataFrame(labels, columns = ['Frame_id','Face?'])#creates a new Pandas DataFrame called `labelled` from the `labels` list. The DataFrame has two columns named "Frame_id" and "Face" that correspond to the two values stored in each list within the `labels` list.

labelled.to_csv("text_fileFacehuman_DLCds.csv")#saves the `labelled` DataFrame as a CSV file with the filename "text_fileFacehumands.csv"

print("Done")
# Release resources
cap.release()
# out.release()
cv2.destroyAllWindows()
#return frame_ids   